In [1]:
from utils.binaries import *
from utils.plotting import *

07:07:14 (  +57.1s) [INFO   ] -- import logging
07:07:14 (  +157ms) [INFO   ] -- import numpy as np
07:07:14 (   +11ms) [INFO   ] -- import uncertainties
07:07:14 (  +355ms) [INFO   ] -- import pandas as pd
07:07:14 (    +2ms) [INFO   ] -- import binaries.tools as tools
07:07:15 (  +711ms) [INFO   ] -- import matplotlib.pyplot as plt
07:07:15 (  +468ms) [INFO   ] -- import seaborn as so
07:07:15 (    +7ms) [DEBUG  ] -- font size set to 9.5
07:07:15 (    +1ms) [DEBUG  ] -- label size set to 13.0
07:07:15 (    +1ms) [DEBUG  ] -- figure size set to [6.6, 3.3]
07:07:15 (    +1ms) [DEBUG  ] -- markersize set to 2.0
07:07:15 (    +0ms) [DEBUG  ] -- usetex set to False
07:07:15 (    +2ms) [INFO   ] -- import plotting.tools as plot


In [2]:
import sys

sys.path.append("/cr/data01/filip/fd-pyutils/")

In [3]:
from fdpyutils.fdEye.fdTools import getPixelVector


def get_col_row(pixel_number):

    col = int(np.ceil(pixel_number / 22.0))
    row = int(pixel_number - 22 * (col - 1))

    return row, col


def get_pixel_vector(site, telescope, pixel_number):
    return np.array(getPixelVector(site, telescope, *get_col_row(pixel_number)))


def calculate_pixel_distance(pixel1: tuple, pixel2: tuple):
    t1, p1, t2, p2 = *pixel1, *pixel2
    d1 = get_pixel_vector("Heat", t1, p1)
    d2 = get_pixel_vector("Coihueco", t2, p2)

    return np.linalg.norm(d1 - d2)


def get_closest_pixel(eye, pixel_number):
    minimal_distance, min_tel, min_pix = np.inf, np.nan, np.nan

    for telescope in range(1, 7):
        for pixel in range(1, 441):
            this_distance = calculate_pixel_distance(
                (eye, pixel_number), (telescope, pixel)
            )
            if this_distance < minimal_distance:
                min_tel, min_pix = telescope, pixel
                minimal_distance = this_distance

    return min_tel, min_pix

In [4]:
from utils.Auger.FD import PixelPlot
from matplotlib.gridspec import GridSpec
import pickle

highlights = [[] for _ in range(6)]
heat_highlights = [[] for _ in range(3)]

for cotel in range(1, 4):
    for pix in range(1, 441):
        itel, ipix = get_closest_pixel(cotel, pix)
        highlights[itel - 1].append(ipix)
        heat_highlights[cotel - 1].append((itel, ipix))

    with open(f"heat{cotel}_CoOverlap.pkl", "wb") as heat_pkl:
        pickle.dump(heat_highlights, heat_pkl)

In [ ]:
fig = plt.figure()
gs = GridSpec(
    1,
    6,
    figure=fig,
)
gs.update(left=0.05, right=0.95, wspace=0.05, hspace=0.02)

for cotel in range(1, 7):
    ax = fig.add_subplot(gs[-cotel])
    PixelPlot([np.nan for _ in range(440)], ax=ax, markpixels=highlights[cotel - 1])
    ax.set_title(cotel)

In [ ]:
plot.save(fig, f"XY-heat/HeCoOverlap/heat{cotel}.png")

In [ ]:
PixelPlot([np.nan for _ in range(440)], markpixels=range(1, 440, 22))

In [ ]:
with open("heat1_CoOverlap.pkl", "rb") as heat_pkl:
    l = pickle.load(heat_pkl)

In [ ]:
print(l)